In [20]:
import pandas as pd
import numpy as np

In [35]:
# Load the GME stock dataset
df = pd.read_csv(r"C:\Users\saura\Downloads\Download Data - STOCK_US_XNYS_GME.csv")
df.head(2)

,Date,Open,High,Low,Close,Volume
0,05/01/2023,19.16,19.25,18.46,18.55,"2,356,640"
1,04/28/2023,18.92,19.72,18.88,19.29,"2,836,576"


In [22]:
# Preprocess the dataset
# Perform any necessary data cleaning, feature engineering, or normalization

# Define the state representation and action space
def get_state(observation):
    # Define the state representation for a given observation
    state = [observation['Open'], observation['Close'], observation['Volume']]
    return state


In [23]:
action_space = ['Buy', 'Sell', 'Hold']

In [24]:
# Initialize the Q-table
state_space = [i for i in range(len(df))]
action_space = [i for i in range(len(action_space))]

q_table = np.zeros((len(state_space), len(action_space)))

In [25]:
# Define the Q-learning algorithm
def q_learning(state, action, reward, next_state, alpha, gamma):
    # Update the Q-value for a given state-action pair
    old_q = q_table[state][action]
    next_max_q = np.max(q_table[next_state])
    new_q = (1 - alpha) * old_q + alpha * (reward + gamma * next_max_q)
    q_table[state][action] = new_q

In [26]:
# Train the Q-learning agent
epsilon = 0.9
gamma = 0.95
alpha = 0.1


In [28]:
for episode in range(100):
    state = 0
    total_reward = 0
    done = False
    actions = []  # Define the actions list
    
    while not done:
        # Choose the action with the highest Q-value or random action with epsilon probability
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.choice(action_space)
        else:
            action = np.argmax(q_table[state])
        
        # Execute the action and observe the next state and reward
        current_price = df.loc[state, 'Close']
        next_price = df.loc[state+1, 'Close']
        
        
        if action == 0:  # Buy
            reward = next_price - current_price
        elif action == 1:  # Sell
            reward = current_price - next_price
        else:  # Hold
            reward = 0
        
        next_state = state + 1
        # Update the Q-table
        q_learning(state, action, reward, next_state, alpha, gamma)
        
        total_reward += reward
        state = next_state
        actions.append(action)  # Append the action to the actions list
        
        if state == len(df) - 1:
            done = True
    
    print(f"Episode {episode}: Total reward = {total_reward}")
    epsilon = 0.9 * epsilon
        

Episode 0: Total reward = 1.120000000000001
Episode 1: Total reward = 3.179999999999996
Episode 2: Total reward = 3.0899999999999963
Episode 3: Total reward = 2.3900000000000077
Episode 4: Total reward = 1.5700000000000003
Episode 5: Total reward = 5.129999999999988
Episode 6: Total reward = 5.429999999999996
Episode 7: Total reward = 2.1099999999999994
Episode 8: Total reward = 3.7200000000000024
Episode 9: Total reward = 3.1899999999999906
Episode 10: Total reward = 7.720000000000002
Episode 11: Total reward = 3.3999999999999986
Episode 12: Total reward = 5.569999999999997
Episode 13: Total reward = 6.769999999999996
Episode 14: Total reward = 7.0399999999999885
Episode 15: Total reward = 5.670000000000002
Episode 16: Total reward = 6.189999999999991
Episode 17: Total reward = 7.640000000000001
Episode 18: Total reward = 7.969999999999999
Episode 19: Total reward = 8.319999999999997
Episode 20: Total reward = 7.199999999999999
Episode 21: Total reward = 8.319999999999997
Episode 22: 

In [30]:
# Create a list of optimal actions based on a benchmark strategy
optimal_actions = []
buy_price = None

for i in range(len(df)-1):
    current_price = df.loc[i, 'Close']
    next_price = df.loc[i+1, 'Close']

    if current_price < next_price:  # Buy if the price is expected to increase
        if buy_price is None:
            optimal_actions.append(0)  # Buy
            buy_price = current_price
        else:
            optimal_actions.append(2)  # Hold
    elif current_price > next_price:  # Sell if the price is expected to decrease
        if buy_price is not None:
            optimal_actions.append(1)  # Sell
            buy_price = None
        else:
            optimal_actions.append(2)  # Hold
    else:  # Hold if the price is expected to remain the same
        optimal_actions.append(2)  # Hold


In [39]:
optimal_actions

[0, 1, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 2]

In [31]:
# Evaluate the performance of the agent
profits = []
buy_price = None
for i in range(len(df)-1):
    current_price = df.loc[i, 'Close']
    next_price = df.loc[i+1, 'Close']
    
    if actions[i] == 0:  # Buy
        buy_price = current_price
    elif actions[i] == 1:  # Sell
        if buy_price is not None:
            profit = current_price - buy_price
            profits.append(profit)
            buy_price = None
    else:  # Hold
        pass

total_profit = sum(profits)
print(f"Total Profit = {total_profit}")

Total Profit = 1.8499999999999943


In [33]:
# Calculate annualized return
trading_days_per_year = 252
total_trading_days = len(df) - 1
annualized_return = (total_profit / total_trading_days) * trading_days_per_year
print(f"Annualized Return = {annualized_return}")

Annualized Return = 24.536842105263084


In [34]:
# Calculate accuracy
correct_predictions = sum(actions[i] == optimal_actions[i] for i in range(len(actions)))
accuracy = correct_predictions / len(actions) * 100
print(f"Accuracy: {accuracy}%")


Accuracy: 57.89473684210527%


Please note that this code is a basic framework and may need to be modified to match the specifics of your stock trading scenario and the GME dataset. Also, remember that predicting and trading stocks is a complex task and using Q-learning alone may not

In [ ]:
#Intern at Pranathi
#Student of DataTrained 
#Date -> 03-05-2023
#Time -> 3:17 